In [1]:
import json
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import re

In [2]:
reviews = []
with open('data/train.jsonl') as fl:
    for i, line in enumerate(fl):
        reviews.append(json.loads(line))
print(len(reviews))
df = pd.DataFrame(reviews)
print(df.shape)
df.head()

5000
(5000, 3)


,label,response,context
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,SARCASM,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...


In [3]:
import re 
def function_clean(text):
    text = re.sub(r"http\S+", "", text) #removing the URL Http
    # Removal of mentions
    text = re.sub("@[^\s]*", "", text)
    # Removal of hashtags
    text = re.sub("#[^\s]*", "", text)
    # Removal of numbers
    text = re.sub('[0-9]*[+-:]*[0-9]+', '', text)
    text = re.sub("'s", "", text)   
    return text

In [4]:
df["context"]= df["context"].str.join(" ") 
df['text'] = df['response'] + df['context']
df['text'] = df['text'].apply(lambda text: function_clean(text))

In [5]:
df.head()

,label,response,context,text
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,A minor child deserves privacy and should be k...,I don't get this .. obviously you do care o...
1,SARCASM,@USER @USER trying to protest about . Talking ...,@USER @USER Why is he a loser ? He's just a Pr...,trying to protest about . Talking about him ...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,Donald J . Trump is guilty as charged . The ev...,He makes an insane about of money from the ...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,Jamie Raskin tanked Doug Collins . Collins loo...,Meanwhile Trump won't even release his SAT s...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,Man ... y ’ all gone “ both sides ” the apocal...,Pretty Sure the Anti-Lincoln Crowd Claimed T...


In [6]:
df['label'] = df['label'].apply(lambda x: np.where(df['label'].unique()== x )[0][0]) #0-1

In [7]:
df.tail(100)

,label,response,context,text
4900,1,@USER @USER @USER I voted for Obama and Trump ...,I stand stronger than anyone in protecting you...,"I voted for Obama and Trump , but , Dafuq f..."
4901,1,@USER @USER @USER Miserable people scrutinize ...,The goal of Barack and Michelle Obama is to be...,Miserable people scrutinize others . Try se...
4902,1,@USER @USER @USER ^ ^ ^ This one believes ever...,"Hard to understand how Giuiliani , who was onc...","^ ^ ^ This one believes everything "" Dear L..."
4903,1,@USER @USER @USER BS justify til the end of ti...,I'm very sorry to have to share this video wit...,BS justify til the end of time but no one w...
4904,1,@USER @USER @USER You didn ’ t bother to read ...,@USER @USER @USER I don ’ t know who that guy ...,You didn ’ t bother to read the Mueller rep...
...,...,...,...,...
4995,1,@USER You don't . I have purchased a lot on Am...,@USER Apologies for the inconvenience you face...,You don't . I have purchased a lot on Amazon ...
4996,1,@USER #Emotions you say 🤔 never knew that I th...,"@USER 🤔 idk tho , I think I ’ m #hungry . But ...",you say 🤔 never knew that I think I ’ m just...
4997,1,"@USER @USER @USER You are so right ... "" Yes !...","@USER @USER @USER Peace to you , and two count...","You are so right ... "" Yes ! is not is no..."
4998,1,@USER @USER @USER Another lazy delusional vote...,Bernie Sanders told Elizabeth Warren in privat...,Another lazy delusional voter who takes the...


In [8]:

X = df["text"]
y = df["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1)

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape #validation

((4000,), (1000,), (4000,), (1000,))

In [10]:
vect_1 = TfidfVectorizer(stop_words = stopwords.words('english'), ngram_range=(1,3)).fit(X_train) #tuning 1, 2, 3, 1-3, 1-4
X_train1 = vect_1.transform(X_train)
X_test1 = vect_1.transform(X_test)
#Vectorizer (deep learning will be different)

In [11]:
XGB_model = XGBClassifier(random_state=1)
XGB_model.fit(X_train1, y_train)

print(f"XG Boost train accuary: {XGB_model.score(X_train1, y_train)}")
print(f"XG Boost test accuary: {XGB_model.score(X_test1, y_test)}")
#1. overfit try different parameters to avoid overfitting
#2. deep learning- LSTM

XG Boost train accuary: 0.96125
XG Boost test accuary: 0.665


In [12]:
#predict test: 1800
test = []
with open('data/test.jsonl') as fl2:
    for i, line in enumerate(fl2):
        test.append(json.loads(line))
print(len(test))
df2 = pd.DataFrame(test)
print(df2.shape)
df2.head()

1800
(1800, 3)


,id,response,context
0,twitter_1,"@USER @USER @USER My 3 year old , that just fi...","[Well now that ’ s problematic AF <URL>, @USER..."
1,twitter_2,@USER @USER How many verifiable lies has he to...,[Last week the Fake News said that a section o...
2,twitter_3,@USER @USER @USER Maybe Docs just a scrub of a...,[@USER Let ’ s Aplaud Brett When he deserves i...
3,twitter_4,@USER @USER is just a cover up for the real ha...,[Women generally hate this president . What's ...
4,twitter_5,@USER @USER @USER The irony being that he even...,"[Dear media Remoaners , you excitedly sharing ..."


In [13]:
df2["context"]= df2["context"].str.join(" ") 
df2['text'] = df2['response'] + df['context']
df2['text'] = df2['text'].apply(lambda text: function_clean(text))

In [14]:
finaltest = df2["text"]

In [15]:
finaltest

0          My  year old , that just finished reading N...
1         How many verifiable lies has he told now ? +...
2          Maybe Docs just a scrub of a coach ... I me...
3         is just a cover up for the real hate inside ...
4          The irony being that he even has to ask why...
                              ...                        
1795       is definitely the best out there . No quest...
1796      Ye let her out run wild and infect  more Chi...
1797       Thanks for that , I would have never known ...
1798       Yes also  this on  with loads of <URL> a  I...
1799       you still need to send the link to the fan ...
Name: text, Length: 1800, dtype: object

In [16]:
finaltest = vect_1.transform(finaltest)

In [18]:
prediction = XGB_model.predict(finaltest)
print(prediction)

[0 0 0 ... 1 0 1]


In [44]:
df_pred = pd.DataFrame(data = prediction, columns=["prediction"])

In [45]:
df_pred

,prediction
0,0
1,0
2,0
3,0
4,0
...,...
1795,0
1796,0
1797,1
1798,0


In [46]:
df_pred["twitter"]= (df_pred.index + 1)
df_pred["twitter_index"]= "twitter_" + df_pred["twitter"].astype(str)
df_pred["result"] = df_pred['prediction'].apply(lambda x: 'SARCASM' if x == 0 else 'NOT_SARCASM')

In [47]:
df_pred

,prediction,twitter,twitter_index,result
0,0,1,twitter_1,SARCASM
1,0,2,twitter_2,SARCASM
2,0,3,twitter_3,SARCASM
3,0,4,twitter_4,SARCASM
4,0,5,twitter_5,SARCASM
...,...,...,...,...
1795,0,1796,twitter_1796,SARCASM
1796,0,1797,twitter_1797,SARCASM
1797,1,1798,twitter_1798,NOT_SARCASM
1798,0,1799,twitter_1799,SARCASM


In [54]:
df_pred[["twitter_index", "result"]].to_csv("answer.txt", header=None, index=None, sep=',', mode='w')